# Setup

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, row_number
from pyspark.sql.window import Window

spark = (SparkSession.builder.appName("cs544")
         .master("spark://main:7077")
         .config("spark.executor.memory", "512M")
         .config("spark.sql.warehouse.dir", "hdfs://main:9000/user/hive/warehouse")
         .enableHiveSupport()
         .getOrCreate())

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/03/24 14:03:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# ! wget https://pages.cs.wisc.edu/~harter/cs639/data/sf.zip
# ! unzip sf.zip

In [3]:
!hdfs dfs -cp sf.csv hdfs://main:9000/sf.csv

In [4]:
df = (spark.read.format("csv")
    .option("header", True)
    .option("inferSchema", True)
    .load("hdfs://main:9000/sf.csv"))
columns = [col(c).alias(c.replace(" ", "")) for c in df.columns]
df.select(columns).write.format("parquet").mode("overwrite").save("hdfs://main:9000/sf.parquet")

23/03/24 14:04:33 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
df = spark.read.format("parquet").load("hdfs://main:9000/sf.parquet")
df.createOrReplaceTempView("calls")

# Demos